
# Final Report
* Wilson Velasco
* Scott Barnett

## NLP Project


NLP Project## Goals
* Construct a ML model that predicts 
* Find key drivers of `FILL`
* Deliver a report to the data science team 
* Deliver a presentation to a general audience

## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from pydataset import data
import scipy.stats as stats
import seaborn as sns
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
import acquire
import wrangle
import functions


## Acquire

* Data acquired from 
    *    .csv files were imported
    
* Original dataframe contained `SHAPE` rows and `SHAPE` columns
* The data was aquired on `23 MAY 2023`
* Each row represents `FILL`
* Each column represents `FILL`


## Prepare

* Prepared data
    * Change column names to remove spaces
    * 
* There were `no null or missing values`
* 
* 
* Outliers were not addressed in this itteration
* Split data into **train**, **validate**, **test**
    


### Data Dictionary

### Bring in Data for Exploration

In [2]:
# Bring in data .csv 

# Split data
train, validate, test = wrangle.split_data(df)

train -> (3897, 14), 59.98%
validate -> (1300, 14),20.01%
test -> (1300, 14), 20.01%


### Sneak Peak at the Data


In [9]:
train.head(3)

NameError: name 'train' is not defined

### Summary of Data

In [8]:
train.describe()

NameError: name 'train' is not defined

## Exploration

* All features independently evaluated against wine quality
* alpha =.05 for all statistics ran
* For categorical features 'T-tests' were used


In [6]:
# viualization distribution of target 


### 1. QUESTION?

In [5]:
# Get visualizaions


#### Charting Observations: 'FILL'
* 

####  Statistics: 'FILL'


##### Hypothisis: 'FILL'
$H_0$ : 'FILL' is independent of 'FILL'<br>
$H_a$ : 'FILL' is not independent of 'FILL'

In [7]:
# Running stat test


#### Result
* Based on the result of our test, we can infer that 

### 2. QUESTION?

In [1]:
# Get visualizaions


#### Charting Observations: 'FILL'

* Higher fluctuation or spread of volatile acidity results in a lower wine quality grade.
* Conversely, lower fluctuation or spread of volatile acidity results in a higher wine quality grade.

#### Result
* Based on the results of our test, we can infer that 

### 3. QUESTION?

In [2]:
# Get visualizaions


#### Charting Observations: 'FILL'
* There is a 

#### Result
* Based on the observation in the chart and the supporting evidence 

### 4. QUESTION?

In [3]:
# Get visualizaions


#### Chart Observations: 'FILL'
* 

#### Result
* Based on our test result, we can infer 

#### Charting Observations: `FILL`
* no clearly observable/useful clusters

## Exploration Summary
* 

### Features we are moving to modeling with
* 
* **'FILL': is our target variable**

### Features not moving to modeling 


## Modeling

### Introduction
* Primary evaluation metric will be **Accuracy**
    * There is no greater value in false positives or false negatives


#### Baseline 

In [20]:
# get baseline 


,Baseline,Metric,Score
0,6.0,Accuracy,0.439569


In [22]:
# running models on cluster set
functions.cluster_model(new_train,y_train)

,model,set,accuracy
0,k_nearest_neighbors,train,0.549397
1,logistic_regression,train,0.544778
2,DecisionTreeClassifier,train,0.539902
3,random_forest,train,0.545548


### Model Selection
* All Models have very similar results
    * All out performed the baseline accuracy of 'FILL'
    * 'FILL'gave the best results on the train and validate sets
* We will use the **'FILL'** model for the unseen Test data

### Test Model: 'FILL'


In [23]:
y_train.sample()

114    5
Name: quality, dtype: int64

In [4]:
# running 'FILL'  on test set


### Modeling Wrap 
* On the test set, the model returned an 'FILL' score of 'FILL'
    * 'FILL' the baseline 
    * Also outperformed the models training and validation runs
* We can reasonably presume that the model will perform consistantly in the same range
* This model **'FILL'** be used in production

## Conclusion

### Summary
* The four features we evaluated closely: 
* Only 
* 
* Our best model was 'FILL' 
    
### Recommendations
* The model provided can be used to 'FILL'

### Next Steps
* If provided more time to work on the project we would want to explore 'FILL'